# Random Forest Classification

In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sn
from numpy import std
from numpy import mean
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## NMI

In [2]:
%cd Z:\PPMI_Data\Excels\Only_Female\GeneCpG\Pain
df = pd.read_csv('NMI_Top50_Pain_Methylome_Female.csv')
print(df.shape)
df.head(3)

Z:\PPMI_Data\Excels\Only_Female\GeneCpG\Pain
(150, 52)


,PATNO,APPRDX,cg20699097,cg12162792,cg25537999,cg03239914,cg10615711,cg17754830,cg20742385,cg20064928,...,cg10644659,cg06512164,cg10387956,cg25696758,cg08592683,cg08596655,cg05226711,cg24282845,cg27262702,cg16919248
0,3002,1,0.522515,0.887998,0.864310,0.024669,0.820693,0.234933,0.831509,0.018605,...,0.913197,0.854714,0.857429,0.877105,0.041617,0.017470,0.534997,0.874573,0.023407,0.854249
1,3003,1,0.504891,0.881857,0.906418,0.027039,0.840767,0.203623,0.804018,0.019957,...,0.896184,0.825011,0.771684,0.884699,0.054625,0.039910,0.606924,0.880717,0.037039,0.840525
2,3006,1,0.540481,0.781014,0.891774,0.034515,0.688983,0.189602,0.838389,0.014110,...,0.883754,0.859714,0.869537,0.856992,0.036285,0.026079,0.549458,0.901012,0.013530,0.878971


In [3]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg20699097,cg12162792,cg25537999,cg03239914,cg10615711,cg17754830,cg20742385,cg20064928,cg03266046,...,cg10644659,cg06512164,cg10387956,cg25696758,cg08592683,cg08596655,cg05226711,cg24282845,cg27262702,cg16919248
0,1,0.522515,0.887998,0.864310,0.024669,0.820693,0.234933,0.831509,0.018605,0.806077,...,0.913197,0.854714,0.857429,0.877105,0.041617,0.01747,0.534997,0.874573,0.023407,0.854249
1,1,0.504891,0.881857,0.906418,0.027039,0.840767,0.203623,0.804018,0.019957,0.817520,...,0.896184,0.825011,0.771684,0.884699,0.054625,0.03991,0.606924,0.880717,0.037039,0.840525


In [4]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [5]:
print(dframe1.head(2))
dframe1.to_csv ("Pain_NMICpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.808571  0.132819
1  0.785714  0.151859


## Boruta

In [6]:
df = pd.read_csv('Boruta_Top50_Pain_Methylome_Female.csv')
print(df.shape)
df.head(3)

(150, 52)


,PATNO,APPRDX,cg02232418,cg05037024,cg01248375,cg01836011,cg26950867,cg04615964,cg25547939,cg19485268,...,cg05391255,cg01806972,cg12225712,cg04281694,cg01378481,cg01162915,cg22148389,cg01354923,cg13107682,cg01947036
0,3002,1,0.830685,0.872853,0.031342,0.784925,0.756753,0.909899,0.915858,0.033116,...,0.886864,0.192149,0.922467,0.879178,0.897877,0.827346,0.921198,0.051568,0.061848,0.017338
1,3003,1,0.768925,0.927371,0.039883,0.799706,0.657526,0.933410,0.910504,0.030570,...,0.918526,0.181933,0.927884,0.878964,0.913737,0.763204,0.916572,0.030401,0.089608,0.012522
2,3006,1,0.785406,0.887823,0.047594,0.774990,0.689543,0.914577,0.902726,0.024966,...,0.906954,0.104059,0.920850,0.881077,0.929086,0.843626,0.901352,0.041172,0.104220,0.013350


In [7]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg02232418,cg05037024,cg01248375,cg01836011,cg26950867,cg04615964,cg25547939,cg19485268,cg13763783,...,cg05391255,cg01806972,cg12225712,cg04281694,cg01378481,cg01162915,cg22148389,cg01354923,cg13107682,cg01947036
0,1,0.830685,0.872853,0.031342,0.784925,0.756753,0.909899,0.915858,0.033116,0.131018,...,0.886864,0.192149,0.922467,0.879178,0.897877,0.827346,0.921198,0.051568,0.061848,0.017338
1,1,0.768925,0.927371,0.039883,0.799706,0.657526,0.933410,0.910504,0.030570,0.086322,...,0.918526,0.181933,0.927884,0.878964,0.913737,0.763204,0.916572,0.030401,0.089608,0.012522


In [8]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [9]:
print(dframe1.head(2))
dframe1.to_csv ("Pain_BorutaCpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.708571  0.156388
1  0.685714  0.195876


## BorutaR

In [10]:
df = pd.read_csv('BorutaR_Pain_CpG_Female.csv')
print(df.shape)
df.head(3)

(150, 12)


,PATNO,APPRDX,cg06585977,cg14841337,cg25909697,cg06300424,cg16200116,cg15438447,cg08369141,cg11615758,cg18027781,cg21570220
0,3002,1,0.032079,0.024749,0.665377,0.907264,0.908723,0.395972,0.014446,0.836631,0.882604,0.020883
1,3003,1,0.030978,0.018556,0.595578,0.890552,0.910992,0.450872,0.010963,0.833505,0.909652,0.019385
2,3006,1,0.028498,0.019744,0.540316,0.930725,0.892911,0.399942,0.011248,0.820000,0.890222,0.022905


In [11]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg06585977,cg14841337,cg25909697,cg06300424,cg16200116,cg15438447,cg08369141,cg11615758,cg18027781,cg21570220
0,1,0.032079,0.024749,0.665377,0.907264,0.908723,0.395972,0.014446,0.836631,0.882604,0.020883
1,1,0.030978,0.018556,0.595578,0.890552,0.910992,0.450872,0.010963,0.833505,0.909652,0.019385


In [12]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [13]:
print(dframe1.head(2))
dframe1.to_csv ("Pain_BorutaRCpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.620000  0.177534
1  0.654286  0.185362


## Common

In [14]:
df = pd.read_csv('Common_Pain_CpG_Female.csv')
print(df.shape)
df.head(3)

(150, 3)


,PATNO,APPRDX,cg08369141
0,3002,1,0.014446
1,3003,1,0.010963
2,3006,1,0.011248


In [15]:
df1 = df.drop(['PATNO'], axis=1)
df1.head(2)

,APPRDX,cg08369141
0,1,0.014446
1,1,0.010963


In [16]:
np.random.seed (1)
dframe1 = pd.DataFrame(columns=['Accuracy', 'Std_Dev'])
for i in range (1,1001) :
        print(i)
        dataset = df1.groupby('APPRDX').apply(lambda x: x.sample(35))
        X = dataset.iloc[:, 1:].values
        y = dataset.iloc[:, 0].values
        classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 1)
        cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=1)
        n_scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
        dframe1 = dframe1.append({'Accuracy': mean(n_scores), 'Std_Dev': std(n_scores)}, ignore_index=True)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [17]:
print(dframe1.head(2))
dframe1.to_csv ("Pain_CommonCpG_Female_1000RF.csv")

   Accuracy   Std_Dev
0  0.494286  0.187812
1  0.417143  0.171047
